## Motivation
While normally I am an *extremely* private person, and do not discuss my *personal sexual preferences* (i.e. what I find attractive, what turns me on, what I find sexually satisfying or erotic), I felt it was necessary to be *brutally* honest (even *graphic*) about the topic of sex/sexuality. After reading this post, it will be virtually impossible to be confused, misled, or unclear about my sexual side.

In [ ]:
# libs for shrinking image size
import os
from PIL import Image
from typing import Optional

def shrink_image_to_size(
    input_path: str,
    output_path: str,
    target_size_kb: int,
    step: int = 5
) -> None:
    """Shrinks the image to approximately the target size by adjusting its quality."""
    # open image using PIL
    img: Image.Image = Image.open(input_path)

    # convert the image to RGB mode if it's not already
    if img.mode != 'RGB':
        img = img.convert('RGB')
    
    # start with a high quality setting
    quality: int = 95

    # Keep reducing quality until the file size is below the target
    while True:
        # Save the image with the current quality
        img.save(output_path, 'JPEG', quality=quality)

        # Check the file size in KB
        file_size_kb: float = os.path.getsize(output_path) / 1024

        # Break the loop if the file size is within the desired range
        if file_size_kb <= target_size_kb:
            break
        
        # Reduce the quality for the next iteration
        quality -= step

        # If quality becomes too low, stop
        if quality < 10:
            print(
                "Warning: Unable to reduce image below target size "
                "without significant quality loss."
            )
            break

def shrink_images_in_directory(
    directory: str,
    target_size_kb: int,
    output_dir: Optional[str] = "resized_images"
) -> None:
    """Shrinks all images in a directory to the target size by adjusting the quality."""
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(directory):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            input_path: str = os.path.join(directory, filename)
            output_path: str = os.path.join(output_dir, filename)

            shrink_image_to_size(input_path, output_path, target_size_kb)

In [ ]:
# libs to open dir and encode
import base64
from typing import List, Tuple

def encode_images_to_base64(
    image_dir: str,
    supported_formats: Tuple[str, ...] = ('jpg', 'jpeg', 'png'),
)-> List[str]:
    """Encode all images in the specified directory to base64 strings."""
    # store b64 string
    encoded_images: List[str] = []
    
    # loop through each file in the directory
    for filename in os.listdir(image_dir):
        # check if the file has the correct extension
        if filename.endswith(supported_formats):
            # get path
            image_path = os.path.join(image_dir, filename)
            
            # open the image file in binary mode
            with open(image_path, "rb") as image_file:
                # read the file and encode it in base64
                encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
                
                #add the encoded string to the list
                encoded_images.append(encoded_string)

    # get list of encoded images
    return encoded_images


In [ ]:
# get necessary libs
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO

# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

def crop_to_target(image, target_size):
    """Crop the image to fit the target size from the top down."""
    width, height = image.size
    target_width, target_height = target_size

    # Resize image with aspect ratio preservation
    if width / height > target_width / target_height:
        new_height = target_height
        new_width = int((width / height) * new_height)
    else:
        new_width = target_width
        new_height = int((height / width) * new_width)
    
    image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
    
    # Ensure image is in RGB format for further processing
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    # Calculate cropping box from the top
    left = (new_width - target_width) / 2
    top = 0
    right = (new_width + target_width) / 2
    bottom = target_height
    
    # Ensure cropping box is within image bounds
    if right > new_width:
        right = new_width
        left = right - target_width
    if bottom > new_height:
        bottom = new_height
        top = bottom - target_height

    image = image.crop((left, top, right, bottom))
    image = image.resize(target_size, Image.Resampling.LANCZOS)
    return image

def display_images_in_grid(
    encoded_images: List[str],
    titles: List[str], 
    target_size: Tuple[int, int],
    rows: int,
    bottom_title: Optional[str] = None,
    bottom_title_coords: Optional[Tuple[float, float]] = (0.5, 0.15),
    ax_title_y_coord: Optional[Tuple[float, float]] = -0.15,
    fig_size: Optional[Tuple[int, int]] = (5, 5),
    bg_color: str = "#181818"
) -> None:
    """Display base64 decoded images in a grid using matplotlib."""
    # get some initial info
    num_images: int = len(encoded_images)
    cols: int = num_images // rows + (num_images % rows > 0)  # Calculate number of columns
    
    # create the subplots grid
    fig, axs = plt.subplots(nrows=rows, ncols=cols, figsize=fig_size)
    
    # set the background color of the figure and axes
    fig.patch.set_facecolor(bg_color)

    # flatten axs if there is only one row or column
    if isinstance(axs, np.ndarray):
        for ax in axs.flat:
            ax.set_facecolor(bg_color)
        axs = np.ravel(axs)

    else:
        axs = [axs]

    # loop through each encoded image, decode, resize, and display
    for i, ax in enumerate(axs):
        # ensure we don't go out of bounds
        if i < num_images:
            # decode the base64 string back into an image
            image_data = base64.b64decode(encoded_images[i])
            image = Image.open(BytesIO(image_data))
            
            # resize the image while preserving the aspect ratio and cropping
            image = crop_to_target(image, target_size)
            
            # determine colormap based on image mode
            if image.mode == 'RGB':
                ax.imshow(image)
            else:
                ax.imshow(image, cmap='gray')

            # set the subtitle (title) for each image
            if i < len(titles):
                ax.set_title(titles[i], color="white", y=ax_title_y_coord)
            
            # turn off axis for cleaner display
            ax.axis('off')
        else:
            # hide empty subplots
            ax.axis('off')

    # add a bottom title if provided
    if bottom_title:
        fig.text(
            *bottom_title_coords,  # Center the text at the bottom
            bottom_title, 
            ha='center', 
            color="white",            
        )
    
    # adjust layout for better spacing between images
    plt.tight_layout()
    plt.show()

## 1 Picture == 1,000 Words
While I am *heterosexual*, I think the best way to really *"communicate"* what I find erotic is through images:

In [ ]:
# storing b64 images for censored erotic mosaic
mosaic = [
    '...',
]

# display
display_images_in_grid(
    mosaic,
    [""],
    (400, 400),
    1,
    "Figure 1. Censored Erotic Samples Mosaic",
    (0.5, -0.05),
    -0.2,
    (5, 5),
)

In *figure 1* above you can see a **censored** collection of images (actually it's an *image mosaic* but it's not obvious that the underlying image is [this](https://www.instagram.com/msblairewhite/p/C8Fp_D4JGNM/)) that serves as a *sample* of what I find **erotic**. If you would like to see the *uncensored* **NSFW** mosaic click [here](https://ch4dth0nd3rk0k.imgbash.com/images/136863.html?fromFolderId=41755) or if you want to see the **NSFW** individual images that comprise the mosaic click [here](https://ch4dth0nd3rk0k.imgbash.com/folders/erotic/).

I picked these images at random from videos that I like. I didn't think hard about it, or second guess it, I was just honest about what I find erotic. So when I say these images capture what I find erotic, it's *unabashedly true*. Understand that I am not overly fond of sharing this part of myself, and that the only reason I am doing this is to be completely transparent. I believe this transparency is necessary for not only someone who may be potentially interested in me personally, but also for my own self-understanding.